# Using `Sage` for Computing Stuff

In [1]:
%display latex
latex.matrix_delimiters("[", "]")
from sage.manifolds import *

## Canonical Forms in Linear Algebra

Given a matrix $A$, we wish to compute its 
- (right) Kernel $\ker A$  

- Characteristic polynomial $c_A(x)$  

- Minimal polynomial $m_A(x)$  

- Eigenvalues&eigenvectors   

- Jordan Canonical Form $J$ and $T$, such that $J=T^{-1}AT$  

- Rational canonical form $C$

### Matrix Input  

It's desirable to copy and past the matrix from a PDF document. We can compute the aforementioned things following the steps below.  
- Copy and paste the matrix into the `input`
- Run the cells sequentially  

The following example is taken from Term1 PS7 Q3.  

In [36]:
print("Enter the array:\n")   

userInput0 = input().splitlines()

myseq = userInput0[0]

myseq=myseq.replace('−', '-')

myseq1 = myseq.split(' ')

size=len(myseq1)

n=ceil(sqrt(size))

myseq2 = [myseq1[i:i+n] for i in range(0,size-1,n)]

#This is the final 2D array
myseq3 = [list(map(int, i)) for i in myseq2]
A = matrix(myseq3)
print(A)

Enter the array:

0 1 0 1 0 0 0 1 1 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0
[0 1 0 1 0]
[0 0 1 1 0]
[0 0 0 0 1]
[0 0 0 0 1]
[0 0 0 0 0]


Computing the **kernel**

In [37]:
A.right_kernel()

Free module of degree 5 and rank 2 over Integer Ring
Echelon basis matrix:
[ 1  0  0  0  0]
[ 0  1  1 -1  0]

### **Computing the characteristic polynomial and its irreducible factors**

In [38]:
c = A.characteristic_polynomial()
c

x^5

In [39]:
A.characteristic_polynomial().factor()

x^5

### **Computing the eigenvalues&eigenvectors**

In [40]:
A.eigenspaces_right()

[
(0, Vector space of degree 5 and dimension 2 over Rational Field
User basis matrix:
[ 1  0  0  0  0]
[ 0  1  1 -1  0])
]

In [41]:
A.eigenvectors_right()

[(0,
  [
  (1, 0, 0, 0, 0),
  (0, 1, 1, -1, 0)
  ],
  5)]

In [42]:
A.eigenmatrix_right()

(
[0 0 0 0 0]  [ 1  0  0  0  0]
[0 0 0 0 0]  [ 0  1  0  0  0]
[0 0 0 0 0]  [ 0  1  0  0  0]
[0 0 0 0 0]  [ 0 -1  0  0  0]
[0 0 0 0 0], [ 0  0  0  0  0]
)

### **Computing the minimal polynomial**

In [43]:
m = A.minimal_polynomial()
m

x^4

In [44]:
m.factor()

x^4

In [45]:
# Comparing with c(x)
c

x^5

### **Computing the JCF**

In [46]:
# use sympy instead
from sympy import Matrix
import numpy as np

In [47]:
T,J = Matrix(A).jordan_form() # using Sympy

In [48]:
T = np.array(T)
T

array([[2, 1, 0, 0, 0],
       [0, 2, 0, 0, -1],
       [0, 0, 1, 0, -1],
       [0, 0, 1, 0, 1],
       [0, 0, 0, 1, 0]], dtype=object)

In [49]:
J = np.array(J)
J

array([[0, 1, 0, 0, 0],
       [0, 0, 1, 0, 0],
       [0, 0, 0, 1, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0]], dtype=object)

### Computing the JCF in Finite Fields

In [53]:
J,T = A.jordan_form(FiniteField(2),transformation=True) # using Sage

In [54]:
T

[1 0 0 0 0]
[0 1 0 1 0]
[0 0 1 1 1]
[0 0 0 1 0]
[0 0 0 0 1]

In [55]:
J

[0 1 0|0 0]
[0 0 1|0 0]
[0 0 0|0 0]
[-----+---]
[0 0 0|0 1]
[0 0 0|0 0]

In [57]:
T**(-1)*J*T # verify

[0 1 0 1 0]
[0 0 1 1 0]
[0 0 0 0 1]
[0 0 0 0 1]
[0 0 0 0 0]

### **Computing the RCF**

In [58]:
A = matrix(QQ,A)
C = A.rational_form(format='right')
C

[0|0 0 0 0]
[-+-------]
[0|0 0 0 0]
[0|1 0 0 0]
[0|0 1 0 0]
[0|0 0 1 0]

## Plotting Differential Equations

It is often desirable to plot the system of equations when solving problems about stability analysis in the 2-D plane.  
Here is a simple example of a two dimensional system from problem sheet week 8.  

$$
\begin{aligned}
\dot{x} &= -x+y-x^3 \\
\dot{y} &= -x-y-y^3
\end{aligned}
$$  

In [21]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import odeint

lim = 3 # upper and lower limits of initial values
N = 1000 # no. of time slices, higher is (ususally) more accurate and makes the plot smoother

def du_dt(u, t):
    # Defines the system of equations
    '''
    - u: 2-D vector
    - t: 1-D real
    '''
    x = u[0]
    y = u[1]
    
    # 2-dim nonlinear system  
    dxdt = -x + y - x**3
    dydt = -x - y - y**3
    
    return [dxdt, dydt]

# Trajectories in forward time.

ts = np.linspace(0, 100, N)

I = np.linspace(-lim, lim, 10) # range of initial values I*I interval in R^2

for x0 in I:
    # outer loop is the initial values of x
    for y0 in I:
        # inner loop is the initial values of y
        u0 = [x0, y0]
        xs = odeint(du_dt, u0, ts) # using built-in methods for integrating
        plt.plot(xs[:,0], xs[:,1], "b-") #  plot
        
#######################################################################################################
#######################################################################################################
# Now plot the vector field  

lim = 1 # limits of x, y values
n = 20 # no. of spacings, higher is denser
x, y = np.meshgrid(np.linspace(-lim, lim, n), np.linspace(-lim, lim, n))

dxdt = -x + y - x**3
dydt = -x - y - y**3

plt.quiver(x,y,dxdt,dydt, color = 'r')

fig = plt.gcf() # get figure
fig.set_size_inches(10,5) # set figure size
fig.set_dpi(300) # set figure resolution

# labelling
plt.xlabel("x", fontsize=15) 
plt.ylabel("y", fontsize=15)
plt.tick_params(labelsize=15)

# setting limits for x-y axis
plt.xlim(-lim, lim)
plt.ylim(-lim, lim)

show(plt)

<module 'matplotlib.pyplot' from '/opt/sagemath-9.3/local/lib/python3.7/site-packages/matplotlib/pyplot.py'>

<Figure size 3000x1500 with 1 Axes>